Now that we have seen how we can use vectors to calculate the trajectory of a ball acting under the influence of gravity, we can move on to something more exotic.  In this exercise, we will incorporate wind resistance (or drag force) acting on the ball into our simulation and the magnus effect, which controls how the spin of the ball effects the path.  

First we begin with the drag force acting on the ball.  Drag is type of friction force that acts on a moving body in the direction opposite the direction of motion of the body when the body is in a fluid.  The underlining causes of drag are a little complicated to discuss in this context, but the equation is relatively easy to write down.  In one dimension the magnitude of the drag force is given by 

$F_D = 0.5\rho v^2C_DA$

where $\rho$ is the density of the surrounding fluid--in this case air, $v$ is the speed,$C_D$ is the dimensionless drag coefficient, and $A$ is the cross section area.  The density of air is usually around $1.225\times 10^{-6}\space kg/cm^3$.  The cross section area of the ball is just $\pi r^2$, where the radius of a typical soccer ball is around $11\space cm$. According to NASA, the coefficient of drag for a typical soccer ball is $0.25$, which is good enough for me (they put men on the moon!).  Can you write the vector equivalent of the drag force in terms of the ball's velocity at a given point of time? 

We shall use the velocity vector ${\bf{v}}(t)$ to indicate the velocity at a given point in time.  The free-body diagram is given below for an arbitrary velocity of the center of the ball (blue arrow)


In [34]:
import numpy as np



#initial positions array for vertex of
a = (1+np.sqrt(5))/2
pos = np.array([[0,1, 3*a],[0,-1,-3*a],[0,1, -3*a],[0,-1,3*a],
[3*a,0,1],[-3*a,0,-1],[3*a,0,-1],[-3*a,0,1],
[1, 3*a,0],[-1,-3*a,0],[1, -3*a,0],[-1,3*a,0],
[1,(2+a), 2*a],[-1,-(2+a), -2*a],[1,-(2+a), -2*a],[-1,(2+a), 2*a],[-1,-(2+a), 2*a],[1,(2+a), -2*a],[1,-(2+a), 2*a],[-1,(2+a), -2*a],
[2*a, 1, (2+a)],[-2*a, -1,-(2+a)],[2*a, -1, -(2+a)],[-2*a, 1,(2+a)], [-2*a, -1, (2+a)],[2*a, 1,-(2+a)],[2*a, -1, (2+a)],[-2*a, 1,-(2+a)],           
[(2+a),2*a,1],[-(2+a),-2*a,-1],[(2+a),-2*a,-1],[-(2+a),2*a,1],[-(2+a),-2*a,1],[(2+a),2*a,-1],[(2+a),-2*a,1],[-(2+a),2*a,-1],
[2,(1+2*a), a], [-2,-(1+2*a), -a], [2,(1+2*a), -a],[-2,-(1+2*a), a],  [2,-(1+2*a), a],[-2,(1+2*a), -a],[2,-(1+2*a), -a],[-2,(1+2*a), a],
[(1+2*a),a, 2], [-(1+2*a),-a, -2], [(1+2*a),-a, -2], [-(1+2*a),a, 2],[(1+2*a),a, -2], [-(1+2*a),-a, 2],[-(1+2*a),a, -2], [(1+2*a),-a, 2],
[a,2, (1+2*a)], [-a,-2, -(1+2*a)], [a,2, -(1+2*a)], [-a,-2, (1+2*a)], [a,-2, -(1+2*a)], [-a,2, (1+2*a)], [a,-2, (1+2*a)], [-a,2, -(1+2*a)]])

#radius of ball in cm
radiusBall = 11
#scale to correct radius
pos = radiusBall/5.0* pos 

#seconds
timeInterval = 1.5
#seconds
dt = 0.001

numSteps = int(np.ceil(timeInterval/dt))
#revolutions per minute
rpm = 3000
#direction of angular velocity vector; no need to be a unit vector
angVelDirection = np.array([0,0,1])

#mass in kg
mass = 0.43

angularVelocity = rpm/60.0*2*np.pi*angVelDirection/np.linalg.norm(angVelDirection)
initPos = np.array([0.0,0.0,0.0])
initVelocity = np.array([0.0,2600.00, 1000.00])
# kg/cm*3
airDens = 1.225e-6
gravityOn = True
windResistanceOn = True
magnusEffectOn = True
#acceleration due to gravity in cm/s^2
gravity = np.array([0.0,0.0,-981.00])
dragCoeff = 0.25
crossSectArea = np.pi*radiusBall**2

def calculateAcceleration(velocity):
    acceleration = np.array([0.0,0.0,0.0])
    if gravityOn == True:
        acceleration += gravity
    if windResistanceOn ==True:
        #D = .5 * rho * V^2 * A * cd
        acceleration += -0.5* airDens*np.linalg.norm(velocity)* crossSectArea*dragCoeff/mass*velocity
    if magnusEffectOn==True:
        acceleration +=np.pi**2*radiusBall**3*airDens/mass*np.cross(angularVelocity,velocity)
    return acceleration
def integratePosition(position, velocity, acceleration):
        position = position+ velocity * dt + 0.5* acceleration*dt**2
        return position
def integrateVelocity(velocity, acceleration):
    newVelocity = velocity + 0.5* acceleration*dt
    acceleration = calculateAcceleration(velocity)
    newVelocity = newVelocity + 0.5* acceleration*dt
    return newVelocity, acceleration

def calculateRotationMatrixVectors(angVel):
    transMat = np.zeros((3,3))
    #we calculate each column of the rotation matrix 
    x = np.array([1.0,0.0,0.0])
    y = np.array([0.0,1.0,0.0])
    z = np.array([0.0,0.0,1.0])
    
    VecList = []
    #first column (spin is counter clockwise)
    projVec = np.dot(x,angVel)/np.linalg.norm(angVel)**2*angVel
    projVecLength = np.linalg.norm(projVec)
    firstVec = x -projVec
    firstVecLength = np.linalg.norm(firstVec)
    if firstVecLength < 1e-6:
        scaledSecVec = np.zeros(3)
    else:
        secondVec = np.cross(angVel,firstVec)
        scaledSecVec = firstVecLength* secondVec/np.linalg.norm(secondVec)
    VecList.append( [projVec, firstVec, scaledSecVec])
    
    #second column (spin is counter clockwise)
    projVec = np.dot(y,angVel)/np.linalg.norm(angVel)**2*angVel
    projVecLength = np.linalg.norm(projVec)
    firstVec = y -projVec
    firstVecLength = np.linalg.norm(firstVec)
    if firstVecLength < 1e-6:
        scaledSecVec = np.zeros(3)
    else:
        secondVec = np.cross(angVel,firstVec)
        scaledSecVec = firstVecLength* secondVec/np.linalg.norm(secondVec)
    VecList.append( [projVec, firstVec, scaledSecVec])
    
    #third Column (spin is counter clockwise)
    projVec = np.dot(z,angVel)/np.linalg.norm(angVel)**2*angVel
    projVecLength = np.linalg.norm(projVec)
    firstVec = z -projVec
    firstVecLength = np.linalg.norm(firstVec)
    if firstVecLength < 1e-6:
        scaledSecVec = np.zeros(3)
    else:
        secondVec = np.cross(angVel,firstVec)
        scaledSecVec = firstVecLength* secondVec/np.linalg.norm(secondVec)
    VecList.append( [projVec, firstVec, scaledSecVec])
    return VecList

def calculateBallPositions(time,vecList, angVel, pos):
    w = np.linalg.norm(angVel)
    transMat = np.zeros((3,3))
    for i, vList in enumerate(vecList):
        transMat[:,i]= vList[0]+vList[1]*np.cos(w*time)+ vList[2]*np.sin(w*time)
        
    for i in range(np.shape(pos)[0]):
        pos[i,:] = np.dot(transMat, pos[i,:])
        
    return pos

#### now we actually run the simulation ####

vectorList = calculateRotationMatrixVectors(angularVelocity)
acceleration = calculateAcceleration(initVelocity)
position = initPos
velocity= initVelocity

print np.linalg.norm(velocity)
handle = open('full.xyz', 'w')
for i in range(numSteps):
    position = integratePosition(position, velocity, acceleration)
    velocity, acceleration= integrateVelocity(velocity, acceleration)
    time = i * dt
    newPos = calculateBallPositions(time, vectorList, angularVelocity, pos)
    newPos = pos+position
    s = ''
    s += str(60)+ '\n\n'
    for j in range(np.shape(pos)[0]):
        s += '%-3s %10.5f %10.5f %10.5f\n' % ('c', newPos[j,0],newPos[j,1],newPos[j,2])
    handle.write(s)
handle.close()
print np.linalg.norm(velocity)
    
    

2785.67765544
2079.58987257
